In [ ]:
!pip install pyngrok
!pip install streamlit
!pip install streamlit-bokeh

import streamlit as st
from pyngrok import ngrok
import os
import subprocess
import time

!ngrok config add-authtoken 2u3FsgMopgpXIrOIFaITVO4ybUj_4Nah2nRGLS76NZoTZnxFN

def run_streamlit():
    proc = subprocess.Popen(["streamlit", "run", "index.py" , "--server.port=8501"])
    time.sleep(5)  # Give Streamlit time to start
    return proc

open("index.py","w").write("""
import streamlit as st

main_page = st.Page("Culinary_Compass_Recommendation_Tool.py")
page_2 = st.Page("Visualizations.py")
page_3 = st.Page("Data_Visualizations.py")
pages = st.navigation([main_page, page_3])
pages.run()
""")

open("Culinary_Compass_Recommendation_Tool.py","w").write("""
import streamlit as st
import pandas as pd
import os
import sys
import logging
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

st.set_page_config(page_title = "Recommendation Tool")
st.sidebar.header('Recommentation tool')
st.sidebar.write('This this the recommendation tool. Based on user inputs, it will provide menu item recommendation.')
st.image('dishes.png')
st.header(':blue[Culinary Compass Dishes Recommendation]')
st.subheader(':blue[Select the cuisine and flavor of preference]', divider = True)

menu_item_data = pd.read_excel('Weighted_Menu_Items.xlsx')
menu_item_data['menu_item'] = menu_item_data['menu_item'].fillna('')
cuisine_list = menu_item_data['cuisine'].sort_values().unique()
cuisine_dropdown = st.selectbox('Select your favorite cuisine', cuisine_list, placeholder = 'Select the cuisine type',)
menu_item_data_cuisine = menu_item_data.groupby('cuisine').get_group(cuisine_dropdown)

flavor_list = menu_item_data['dominant_flavor'].sort_values().unique()
flavor_dropdown = st.selectbox('Select your favorite flavor', flavor_list, placeholder = 'Select your flavor',)
choice = st.button('Show me recommendations')

def load_menu_data(file_path: str) -> pd.DataFrame:
  if not os.path.exists(file_path):
      logging.error(f"File not found: {file_path}")
      raise FileNotFoundError(f"The file {file_path} does not exist.")
  logging.info(f"Loading file: {file_path}")
  return pd.read_excel(file_path)

def process_data(df: pd.DataFrame) -> np.hstack:
  df['flavour_weight'] = df['flavour_weight'].apply(eval)
  flavor_df = pd.json_normalize(df['flavour_weight'])
  scaler = StandardScaler()
  scaled_flavors = scaler.fit_transform(flavor_df)
  tfidf = TfidfVectorizer(stop_words='english')
  ingredient_tfidf = tfidf.fit_transform(df['ingredients'])
  combined_features = np.hstack([scaled_flavors, ingredient_tfidf.toarray()])
  return combined_features


def recommend_cosine_sim(menu, cuisine, taste_profile, combined_features, top_n=5):
  taste_profile = taste_profile.strip().lower()
  taste_profile_dic = {taste_profile: 1.0}
  valid_flavors = {"sweet", "salty", "sour", "bitter", "umami"}
  flavor_profile = {flavor: 0.0 for flavor in valid_flavors}
  for key in taste_profile_dic:
        if key in flavor_profile:
            flavor_profile[key] = taste_profile_dic[key]
        else:
            raise ValueError(f"Invalid flavor: {key}. Must be one of {list(valid_flavors)}")
  filtered_df = menu[menu['cuisine'].str.strip().str.lower() == cuisine.strip().lower()].copy()
  filtered_df = filtered_df[filtered_df['dominant_flavor'].str.strip().str.lower() == taste_profile].copy()
  if filtered_df.empty:
    return f"No menu items found for cuisine: {cuisine}"
  flavor_profiles = np.array([
        list(flavor.values()) if isinstance(flavor, dict) else [0] * len(flavor_profile)
        for flavor in filtered_df['flavour_weight']])
  input_profile = np.array(list(flavor_profile.values())).reshape(1, -1)
  cosine_sim = cosine_similarity(input_profile, flavor_profiles).flatten()
  cosine_dist = 1 - cosine_sim
  sim_scores = np.argsort(cosine_dist)[:top_n]
  recommendations = filtered_df.iloc[sim_scores][['menu_item_id', 'menu_item', 'cuisine', 'ingredients', 'dominant_flavor']]
  recommendations['score'] = cosine_dist[sim_scores]
  return recommendations

def get_restaurant_name(name):
  name = name[:-3]
  name = name.replace("_", " ")
  name = name.title()
  return name

def main():
  try:
        path = "Weighted_Menu_Items.xlsx" #Path to menu items
        menu = load_menu_data(path) #Loads excel into df

        cuisine = cuisine_input
        taste_profile = flavor_input
        combined_features = process_data(menu)
        n = 5
        visualize = False
        res_cos = pd.DataFrame(recommend_cosine_sim(menu, cuisine, taste_profile, combined_features, n))
        df_empty = res_cos.empty
        st.write("Based on your choice of cuisine (" + cuisine_dropdown + ") and flavor (" + flavor_dropdown + "), we recommend the following menu items:")
        if (df_empty):
          st.write("No menu items found for your choice.")
        else:
          num_rows = res_cos.shape[0]
          for i in range(num_rows):
            st.write("Choice " + str(i+1) + ": " + res_cos['menu_item'].iloc[i] + " at " + get_restaurant_name(res_cos['menu_item_id'].iloc[i]))
          st.subheader("The full dataframe")
          st.dataframe(res_cos)

  except Exception as e:
      logging.error(f"An error occurred: {e}")
      st.write(logging.error(f"An error occurred: {e}"))
      raise


if(choice):
  cuisine_input = cuisine_dropdown
  flavor_input = flavor_dropdown
  main()

""")

open("Data_Visualizations.py","w").write("""
import streamlit as st

st.sidebar.header('Data Visualizations')
st.sidebar.write('Here is a list of various visualizations generated with the data provided.')
st.header(':blue[Data Visualizations]')
st.subheader(":blue[Cuisines]")
st.image('cuisines.png')
st.subheader(':blue[Top Ingredients]')
st.image('top_ingredientsa.png')
st.subheader(':blue[Taste profiles]')
st.image('taste_profiles.png')
st.subheader(':blue[Restaurant availability]')
st.image('restaurant_availability.png')
""")

open("Visualizations.py","w").write("""
import streamlit as st
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show
from bokeh.palettes import cividis
from bokeh.transform import cumsum, dodge
import math
import ipywidgets as wg
import json
from streamlit_bokeh import streamlit_bokeh


st.set_page_config(page_title = "Visualizations")
st.sidebar.header('Visualizations')
st.sidebar.write('Here is a list of various visualizations generated with the data provided and also the data processed by Richika.')
st.header(':blue[Visualizations with data provided]')

st.subheader('Number of menu items by restaurants')
#importation of the raw data
cuisine_to_ingredient = pd.read_csv('cuisines_to_ingredients_mapping.csv')
taste_profile_ingredients = pd.read_csv('taste_profile_to_ingredients_mapping.csv')
user_inputs = pd.read_csv('user_inputs.csv')
restaurant_details = pd.read_excel("Restaurant Details.xlsx")
restaurant_menu_items = pd.read_excel('Restaurant Menu Items.xlsx')

#filling empty values with NaN
restaurant_details = restaurant_details.fillna('NaN')
restaurant_menu_items= restaurant_menu_items.fillna('NaN')

#restaurants informations
restaurant_name = restaurant_menu_items['restaurant_id'].unique()
restaurant_number = len(restaurant_name)
restaurant_group = restaurant_menu_items.groupby('restaurant_id')
y_axis_menu_items = []
colors_menu_items = cividis(len(restaurant_name))

for i in range(restaurant_number):
    y_axis_menu_items.append(restaurant_group.get_group(restaurant_name[i])['menu_item'].count())
    #print("The number of menu items at " + restaurant_name[i]  + " is "+ str(restaurant_group.get_group(restaurant_name[i])['menu_item'].count()))

source_menu_item = {'Restaurants' : list(restaurant_name), 'Menu Items' : y_axis_menu_items, 'Colors': colors_menu_items}

menu_item_figure = figure(title = "The number of menu items for each restaurant", x_axis_label = "Restaurants", y_axis_label = "Number of menu items", x_range = restaurant_name, width = 1200, height = 400)
menu_item_figure.vbar(x = 'Restaurants', top = 'Menu Items', source = source_menu_item, legend_field = 'Restaurants' , width = 0.5, fill_color = 'Colors')
menu_item_figure.xaxis.major_label_orientation = math.pi/4
menu_item_figure.legend.location = "top_center"
menu_item_figure.legend.orientation = "horizontal"

streamlit_bokeh(menu_item_figure, use_container_width=True)

st.write('')
st.write('')

st.subheader('Number of ingredients by elements')
# about elements and ingredients
ingredient_element = pd.DataFrame(columns = ['Ingredient', "Element"])
element_list = []
ingredient_list = []
for i in range(len(taste_profile_ingredients)):
    element_list.append(taste_profile_ingredients['Element'].iloc[i])
    ingredient_list.append(taste_profile_ingredients['Ingredients'][i].split(', '))

for j in range(len(element_list)):
  for i in range(len(ingredient_list[j])) :
    if j == 0:
       ingredient_element.loc[i] = [ingredient_list[j][i] ,element_list[j]]
    else:
      ingredient_element.loc[len(ingredient_element)] = [ingredient_list[j][i] ,element_list[j]]

#ingredients vs element informations
element_name = ingredient_element['Element'].unique()
element_number = len(element_name)
element_group = ingredient_element.groupby('Element')
y_axis_element_items = []
colors_element_items = cividis(len(element_name))

for i in range(element_number):
    y_axis_element_items.append(element_group.get_group(element_name[i])['Ingredient'].count())

source_element_item = {'Elements' : list(element_name), 'Ingredients' : y_axis_element_items, 'Colors': colors_element_items}

element_item_figure = figure(title = "The number of ingrdients for each element", x_axis_label = "Elements", y_axis_label = "Number of ingredients", x_range = element_name, width = 1200, height = 400)
element_item_figure.vbar(x = 'Elements', top = 'Ingredients', source = source_element_item, legend_field = 'Elements' , width = 0.5, fill_color = 'Colors')
element_item_figure.xaxis.major_label_orientation = math.pi/4
element_item_figure.legend.location = "top_center"
element_item_figure.legend.orientation = "horizontal"

streamlit_bokeh(element_item_figure, use_container_width=True)

st.subheader('Flavour profile of Hummus from LayLak Toronto')
#Generation of a new dataframe containing flavour profile
flavour_profile = pd.DataFrame(columns = ['restaurant_id', 'menu_item','ingredients', 'Sour', 'Sweet', 'Salty', 'Spicy', 'Umami', 'Bitter', 'Savory'])
for i in range(len(restaurant_menu_items)):
  flavour_profile.loc[i] = [restaurant_menu_items['restaurant_id'].iloc[i], restaurant_menu_items['menu_item'].iloc[i], restaurant_menu_items['ingredients'].iloc[i], 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN']

def menu_flavour_profile(menu_item, ingredients):
  item_ingredient_list = []
  item_ingredient_list = ingredients.split(', ')
  global sour, sweet, salty, spicy, umami, bitter, savory
  sour = sweet = salty = spicy = umami = bitter = savory = 0
  #print(len(item_ingredient_list))
  for i in range(int(len(item_ingredient_list))):
    #print(item_ingredient_list[i])
    for j in range(len(ingredient_element)):
      if (item_ingredient_list[i] == ingredient_element.loc[j, 'Ingredient']):
        #print('yes')
        #print(ingredient_element.loc[j, 'Element'])
        match ingredient_element.loc[j, 'Element']:
          case 'Sour':
            sour += 1
          case 'Sweet':
            sweet += 1
          case 'Salty':
            salty += 1
          case 'Spicy':
            spicy += 1
          case 'Umami':
            umami += 1
          case 'Bitter':
            bitter += 1
          case 'Savory':
            savory += 1
  return sour, sweet, salty, spicy, umami, bitter, savory

for i in range(len(flavour_profile)):
  menu_flavour_profile(flavour_profile['menu_item'].iloc[i],flavour_profile['ingredients'][i])
  flavour_profile.loc[i, 'Sour'] = sour
  flavour_profile.loc[i, 'Sweet'] = sweet
  flavour_profile.loc[i, 'Salty'] = salty
  flavour_profile.loc[i, 'Spicy'] = spicy
  flavour_profile.loc[i, 'Umami'] = umami
  flavour_profile.loc[i, 'Bitter'] = bitter
  flavour_profile.loc[i, 'Savory'] = savory

flavours = {'Sour' : flavour_profile['Sour'][0], 'Sweet' : flavour_profile['Sweet'][0], 'Salty' : flavour_profile['Salty'][0], 'Spicy' : flavour_profile['Spicy'][0], 'Umami' : flavour_profile['Umami'][0], 'Bitter' : flavour_profile['Bitter'][0], 'Savory' : flavour_profile['Savory'][0]}
data = pd.Series(flavours).reset_index(name='value').rename(columns={'index': 'flavour'})
data['angle'] = data['value']/data['value'].sum() * 2*math.pi
data['color'] = cividis(len(flavours))

flavour_distribution = figure(height=350, title="Flavour distribution of " + flavour_profile['menu_item'][0], toolbar_location=None, x_range=(-0.5, 1.0))

flavour_distribution.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='flavour', source=data)

flavour_distribution.axis.axis_label = None
flavour_distribution.axis.visible = False
flavour_distribution.grid.grid_line_color = None

#handle = show(flavour_distribution, notebook_handle=True)
#print(flavour_profile['menu_item'])
#print(list(flavour_profile['menu_item']))
#menu_item_select = wg.Dropdown(options = list(flavour_profile['menu_item']), description = 'Select a menu item: ')
#@wg.interact(menu_item = menu_item_select)
#def update(menu_item):
#  for i in range(len(flavour_profile)):
#    if (menu_item == flavour_profile['menu_item'][i]):
#      print(i)
#      flavours = {'Sour' : flavour_profile['Sour'][i], 'Sweet' : flavour_profile['Sweet'][i], 'Salty' : flavour_profile['Salty'][i], 'Spicy' : flavour_profile['Spicy'][i], 'Umami' : flavour_profile['Umami'][i], 'Bitter' : flavour_profile['Bitter'][i], 'Savory' : flavour_profile['Savory'][i]}
#      print(flavours)
#      data = pd.Series(flavours).reset_index(name='value').rename(columns={'index': 'flavour'})
#      data['angle'] = data['value']/data['value'].sum() * 2*math.pi
#      data['color'] = cividis(len(flavours))
#      flavour_distribution.title.text = "Flavour distribution of " + flavour_profile['menu_item'][i] + " at " + flavour_profile['restaurant_id'][i]
#      flavour_distribution.wedge(x=0, y=1, radius=0.4,start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),line_color="white", fill_color='color', legend_field='flavour', source=data)

#  push_notebook(handle = handle)

streamlit_bokeh(flavour_distribution, use_container_width=True)

st.divider()
st.header(":blue[Visualizations with refined data (Based on Richika's work)]")
st.subheader("Flavour distribution of Hummus from Laylak Toronto")
menu_item = pd.read_excel('Weighted_Menu_Items.xlsx')
hummus_loaded = json.loads(menu_item['flavour_weight'][0])

hummus = []
for flavour, weight in hummus_loaded.items():
    hummus.append([flavour ,weight])

hummus_dict = dict(hummus)

data = data = pd.Series(hummus_dict).reset_index(name='value').rename(columns={'index': 'flavour'})
data['angle'] = data['value']/data['value'].sum() * 2*math.pi
data['color'] = cividis(len(hummus_dict))
flavour_distribution = figure(height=350, title="Flavour distribution of " + menu_item['menu_item'][0], toolbar_location=None, x_range=(-0.5, 1.0))

flavour_distribution.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='flavour', source=data)

flavour_distribution.axis.axis_label = None
flavour_distribution.axis.visible = False
flavour_distribution.grid.grid_line_color = None

streamlit_bokeh(flavour_distribution, use_container_width=True)

cuisine  = menu_item.groupby('cuisine')
cuisine_number = cuisine.groups.keys()
colors_cuisine = []
cuisine_menu_item_number =[]
for cuisine_name in cuisine_number:
    cuisine_menu_item_number.append([cuisine_name, len(cuisine.get_group(cuisine_name))])

cuisine_menu_item_number = pd.DataFrame(cuisine_menu_item_number, columns=['cuisine', 'number'])
cuisine_menu_item_number['colors'] = cividis(len(cuisine_number))
cuisine_figure = figure(x_range = list(cuisine_number), height=350, title="Cuisine Distribution", toolbar_location=None)

cuisine_figure.vbar(x='cuisine', top='number', width=0.9, source=cuisine_menu_item_number, fill_color = 'colors')
cuisine_figure.xaxis.major_label_orientation = math.pi/4

st.subheader("Cuisine Distribution")
streamlit_bokeh(cuisine_figure, use_container_width=True)

st.subheader("Flavor Distribution")
dominant_flavour = menu_item.groupby('dominant_flavor')
flavour_number = dominant_flavour.groups.keys()
flavour_menu_item_number =[]

for flavour_name in flavour_number:
    flavour_menu_item_number.append([flavour_name, len(dominant_flavour.get_group(flavour_name))])

flavour_menu_item_number = pd.DataFrame(flavour_menu_item_number, columns=['flavour', 'number'])
flavour_menu_item_number['colors'] = cividis(len(flavour_number))

flavour_figure = figure(x_range = list(flavour_number), height=350, title="Flavour Distribution", toolbar_location=None)

flavour_figure.vbar(x='flavour', top='number', width=0.9, source=flavour_menu_item_number, fill_color = 'colors')
flavour_figure.xaxis.major_label_orientation = math.pi/4

streamlit_bokeh(flavour_figure, use_container_width=True)
""")

def start_ngrok():
    ngrok.kill()  # Kill any existing Ngrok tunnels
    url = ngrok.connect("8501", "http")  # FIXED Port Syntax
    print(f"Public URL: {url}")
    return url

streamlit_proc = run_streamlit()
public_url = start_ngrok()

print(f"Streamlit is running at: {public_url}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 46.2 MB/s eta 0:00:00
  Attempting uninstall: bokeh
    Found existing installation: bokeh 3.6.3
    Uninstalling bokeh-3.6.3:
      Successfully uninstalled bokeh-3.6.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
panel 1.6.1 requires bokeh<3.7.0,>=3.5.0, but you have bokeh 3.7.0 which is incompatible.
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: NgrokTunnel: "https://07d2-34-148-131-21